# Distance-Prior Multinomial NNGP demonstration

このノートブックは、距離先行情報付き多項NNGP Gibbsサンプラーを実行し、事後平均の組成比を可視化します。

- **階層的GP事前分布**: 距離情報を切片の事前平均として組み込む
- 線形予測子: $\eta_k(s) = W(s)^\top \beta_k(s)$
- 切片の事前分布: $\beta_{0k} \sim \text{GP}(\lambda_k \cdot g_k, C)$（距離ベース事前平均）
- 共変量の事前分布: $\beta_{jk} \sim \text{GP}(0, C)$ for $j \geq 1$（標準GP）

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
from matplotlib.colors import TwoSlopeNorm

from bayesian_statistics.models.preprocessing.data_preprocessor import (
    ObsidianDataPreprocessor,
)
from bayesian_statistics.nngp.model import (
    DistancePriorConfig,
    prepare_distance_prior_dataset,
    run_mcmc_with_distance_prior,
)

## データの準備

めも

- lengthscaleは波のゆったりさ
- varianceは波の高さ


| lengthscale | variance | 効果             |
| :---------- | :------- | :--------------- |
| 小          | 大       | 強い局所効果: 局所的なデータに強く反応し、大きく変動 |
| 小          | 小       | 弱い局所効果: 局所的だが変動は小さい |
| 大          | 大       | 強い広域効果: 広域的に滑らかで、大きく変動 |
| 大          | 小       | 弱い広域効果: 広域的に滑らかで、変動は小さい |


In [ ]:
# 現時点でのベスト：各特徴量ごとのパラメータ
lengthscale = {
    "intercept": 0.05,
    "average_elevation": 1,
    "cost_kouzu": 1,
    "cost_shinshu": 1,
    "cost_hakone": 0.1,
    "cost_takahara": 0.1,
}
variance = {
    "intercept": 0.05,
    "average_elevation": 1,
    "cost_kouzu": 1,
    "cost_shinshu": 1,
    "cost_hakone": 0.1,
    "cost_takahara": 0.1,
}

In [ ]:
data_dir = "/home/ohta/dev/bayesian_statistics/data"
preprocessor = ObsidianDataPreprocessor(data_dir, scale_variables=True)
preprocessor.load_data()

period = 2
# 産地は5つ（「その他」をベースラインに）
origins = ["神津島", "信州", "箱根", "高原山", "その他"]

variable_names = [
    # "average_slope_angle",
    "cost_kouzu",
    "cost_shinshu",
    "cost_hakone",
    "cost_takahara",
]

# 各特徴量ごとのパラメータ

# 小さい方が局所的な影響力が強い
lengthscale = {
    "intercept": 0.2,  # 0.2が現時点ベスト
    # "average_slope_angle": 1,
    "cost_kouzu": 1,
    "cost_shinshu": 1,
    "cost_hakone": 0.1,  # 小さめにする
    "cost_takahara": 0.1,
}

# 大きい方が全体的な効果が大きい
variance = {
    "intercept": 0.1,
    # "average_slope_angle": 1,
    "cost_kouzu": 1,
    "cost_shinshu": 1,
    "cost_hakone": 0.1,
    "cost_takahara": 0.1,
}

# prior mean
prior_mean = {
    "intercept": None,  # instead using distance prior
    # "average_slope_angle": 0,
    "cost_kouzu": 0,
    "cost_shinshu": 0,
    "cost_hakone": 0,
    "cost_takahara": 0,
}

# 距離列名（4つの主要産地のみ、cost_riverとその他は除外）
distance_column_names = [
    "cost_kouzu",
    "cost_shinshu",
    "cost_hakone",
    "cost_takahara",
]

# 産地の重要度（4つの主要産地のみ指定、その他は自動的に小さい重みが付与される）
source_weights = [2, 1, 0.05, 0.05]

# λの固定値（K-1=4つの非ベースラインカテゴリに対応）
lambda_fixed = [1, 1, 1, 1]

# グリッド点のサブサンプリング比率
grid_subsample_ratio = 0.01

dataset = prepare_distance_prior_dataset(
    preprocessor=preprocessor,
    period=period,
    origins=origins,
    variable_names=variable_names,
    distance_column_names=distance_column_names,
    grid_subsample_ratio=grid_subsample_ratio,
    drop_zero_total_sites=True,
    tau=0.5,
    alpha=1.0,
    source_weights=source_weights,
)

print(f"Number of sites: {dataset.num_sites()}")
print(f"Number of categories (K): {dataset.num_categories()}")
print(f"Distance features shape: {dataset.distance_features_sites.shape}")
print(f"Number of lambda values: {len(lambda_fixed)}")
print("\nExplanation:")
print("  - K=5 categories: 神津島, 信州, 箱根, 高原山, その他")
print("  - 4 distance columns provided for main sources")
print("  - 'その他' gets dummy distance (far away) automatically")
print(
    f"  - Distance features: K-1={dataset.num_categories() - 1} (one per non-baseline category)"
)

In [ ]:
list(lengthscale.values())

In [ ]:
config = DistancePriorConfig(
    n_iter=200,
    burn_in=50,
    thinning=1,
    neighbor_count=40,
    # kernel_lengthscale=0.1, #0.1
    # kernel_variance=0.1,  #1
    kernel_lengthscale_by_feature=list(lengthscale.values()),
    kernel_variance_by_feature=list(variance.values()),
    prior_mean_by_feature=list(prior_mean.values()),
    tau=1.0,
    alpha=1.0,
    source_weights=source_weights,
    lambda_fixed=lambda_fixed,  # λを固定値として設定
    seed=42,
)

## 距離ベース基準確率の可視化

モデルが使用する距離ベース基準確率 $p_{0k}$ を確認します。

In [ ]:
from bayesian_statistics.nngp.model import weighted_inverse_softmax

boundary = (
    preprocessor.df_elevation.filter(
        pl.col("average_elevation").is_null(), ~pl.col("is_sea")
    )
    .select(["x", "y"])
    .to_numpy()
)

# グリッド上の距離ベース基準確率
# dataset.source_weights_full を使用（ベースライン用の重みを含む5要素）
p0_grid = weighted_inverse_softmax(
    dataset.distance_zscores_grid,
    dataset.source_weights_full,
    tau=dataset.tau,
    alpha=dataset.alpha,
)

# 海洋マスクの作成（dataset.grid_pointsに対応するサブサンプル）
# prepare_multinomial_dataset と同じロジックでサブサンプルインデックスを再生成
elevation_df_sorted = preprocessor.df_elevation.sort(["y", "x"])
n_grid_full = elevation_df_sorted.shape[0]
keep = max(int(np.floor(n_grid_full * grid_subsample_ratio)), 1)
indices = np.linspace(0, n_grid_full - 1, keep, dtype=int)

# is_seaカラムを取得してサブサンプルし、陸地マスクを作成
is_sea_full = (
    elevation_df_sorted.select(pl.col("is_sea").cast(pl.Boolean)).to_numpy().flatten()
)
is_land_subsampled = ~is_sea_full[indices]


def plot_distance_prior(origin_index: int):
    fig, ax = plt.subplots(1, 1, figsize=(8, 6), constrained_layout=True)

    ax.scatter(
        dataset.grid_points[is_land_subsampled, 0],
        dataset.grid_points[is_land_subsampled, 1],
        c=p0_grid[is_land_subsampled, origin_index],
        cmap="Reds",
        s=10,
        alpha=0.8,
    )
    ax.scatter(boundary[:, 0], boundary[:, 1], c="grey", s=0.001)

    plt.colorbar(ax.collections[0], ax=ax, label="baseline probability")
    ax.set_title(f"Distance-based prior: {origins[origin_index]}")
    ax.set_xlabel("longitude")
    ax.set_ylabel("latitude")

    for spine in ax.spines.values():
        spine.set_linewidth(0.5)

    plt.show()


print("距離ベース基準確率（重み付き逆ソフトマックス）:")
print(
    f"使用した重み（K={len(origins)}要素、ベースライン含む）: {dataset.source_weights_full}"
)
# 4つの主要産地について可視化（「その他」は除く）
# for k in range(4):
#    plot_distance_prior(k)

## モデル設定とMCMC実行

### カーネル選択

以下のカーネルタイプから選択できます：
- **"isotropic"**: 標準の等方性RBFカーネル（デフォルト）
- **"distance_dependent"**: 産地からの距離に応じて分散が変化するカーネル

In [ ]:
# ========== カーネル選択 ==========
# "isotropic" または "distance_dependent" を選択
KERNEL_TYPE = "isotropic"  # または "distance_dependent"

# 距離依存分散カーネルのパラメータ（KERNEL_TYPE="distance_dependent"の場合のみ使用）
DISTANCE_SCALING = (
    3  # 距離スケーリング係数gamma（0=等方性、大きいほど遠方の影響範囲が広い）
)
SCALING_TYPE = "linear"  # "linear" または "exponential"
# ===================================

# カーネルの準備
kernel = None
if KERNEL_TYPE == "distance_dependent":
    from bayesian_statistics.nngp.model import DistanceDependentNNGPKernel

    # 全地点（観測点＋グリッド点）の座標を結合
    all_coords = np.vstack([dataset.coords, dataset.grid_points])

    # 各カテゴリ（産地）ごとにカーネルを作成
    # p+1個のカーネル（切片 + 共変量）× K-1個のカテゴリ
    p_plus_1 = dataset.design_matrix_sites.shape[1]  # 切片 + 共変量数
    K_minus_1 = dataset.num_categories() - 1  # 非ベースラインカテゴリ数

    # 各産地への距離Zスコアを結合（観測点＋グリッド点）
    all_distances = np.vstack(
        [dataset.distance_zscores_sites, dataset.distance_zscores_grid]
    )  # (n_all, K-1)

    # 各特徴量について、全カテゴリで共通のカーネルを作成
    # （簡易実装：最初のカテゴリの距離を使用）
    kernels = []
    for j in range(p_plus_1):
        # 最初のカテゴリ（k=0）の距離を使用
        distances_k0 = all_distances[:, 0]

        kernel_j = DistanceDependentNNGPKernel.from_coordinates_and_distances(
            coords=all_coords,
            distances=distances_k0,
            lengthscale=config.kernel_lengthscale,
            base_variance=config.kernel_variance,
            distance_scaling=DISTANCE_SCALING,
            scaling_type=SCALING_TYPE,
        )
        kernels.append(kernel_j)

    kernel = kernels
    print(
        f"Using distance-dependent kernel with gamma={DISTANCE_SCALING}, type={SCALING_TYPE}"
    )
else:
    print("Using isotropic kernel")

# MCMC実行
results = run_mcmc_with_distance_prior(dataset, config, kernel=kernel)
site_probs = results.predict_probabilities(location="sites")
grid_probs = results.predict_probabilities(location="grid", sample_conditional=False)
print(f"Saved samples: {results.beta_samples.shape[0]}")
print(f"Lambda values used: {results.lambda_values}")

## 事後平均の可視化

In [ ]:
time_periods = {0: "早期・早々期", 1: "前期", 2: "中期", 3: "後期", 4: "晩期"}


true_ratio = dataset.counts / dataset.counts.sum(axis=1, keepdims=True)
true_ratio = np.nan_to_num(true_ratio).T


def plot_result(origin_index: int):
    fig, ax = plt.subplots(1, 1, figsize=(8, 6), constrained_layout=True)

    # グリッド上の事後平均
    ax.scatter(
        dataset.grid_points[is_land_subsampled, 0],
        dataset.grid_points[is_land_subsampled, 1],
        c=grid_probs[origin_index, is_land_subsampled],
        cmap="Blues",
        s=10,
        alpha=0.8,
        vmin=0,
        vmax=1,
    )

    # 観測地点の実測比率
    ax.scatter(
        dataset.coords[:, 0],
        dataset.coords[:, 1],
        c=true_ratio[origin_index],
        cmap="Blues",
        s=25,
        edgecolors="black",
        linewidths=0.5,
        alpha=0.8,
        label="observed ratio",
    )
    ax.scatter(boundary[:, 0], boundary[:, 1], c="grey", s=0.001)

    plt.colorbar(ax.collections[0], ax=ax, label="posterior mean")
    ax.set_title(f"{origins[origin_index]}, {time_periods[period]}")
    ax.set_xlabel("longitude")
    ax.set_ylabel("latitude")

    for spine in ax.spines.values():
        spine.set_linewidth(0.5)

    plt.show()

### 神津島

In [ ]:
plot_result(0)

### 信州

In [ ]:
plot_result(1)

### 箱根

In [ ]:
plot_result(2)

### 高原山

In [ ]:
plot_result(3)

In [ ]:
# 効果分解を実行
effects_grid = results.decompose_effects(location="grid")

print("Available effects:")
for key in effects_grid.keys():
    print(f"  - {key}")

## 効果の比較プロット

各効果を比較して可視化します。

In [ ]:
def plot_effect_comparison(origin_index: int, effect_names: list):
    """複数の効果を比較プロット"""
    n_effects = len(effect_names)
    fig, axes = plt.subplots(
        1, n_effects, figsize=(6 * n_effects, 5), constrained_layout=True
    )
    if n_effects == 1:
        axes = [axes]

    for ax, effect_name in zip(axes, effect_names):
        effect_data = effects_grid[effect_name]

        ax.scatter(
            dataset.grid_points[is_land_subsampled, 0],
            dataset.grid_points[is_land_subsampled, 1],
            c=effect_data[origin_index, is_land_subsampled],
            cmap="Blues",
            s=10,
            alpha=0.8,
            vmin=0,
            vmax=1,
        )
        ax.scatter(boundary[:, 0], boundary[:, 1], c="grey", s=0.001)
        plt.colorbar(ax.collections[0], ax=ax, label="probability")
        ax.set_title(f"{origins[origin_index]}: {effect_name}")
        ax.set_xlabel("longitude")
        ax.set_ylabel("latitude")

        for spine in ax.spines.values():
            spine.set_linewidth(0.5)

    plt.show()


# 例：神津島について、距離効果・切片効果・完全モデルを比較
plot_effect_comparison(0, effects_grid.keys())
plot_effect_comparison(1, effects_grid.keys())
plot_effect_comparison(2, effects_grid.keys())
plot_effect_comparison(3, effects_grid.keys())

In [ ]:
effects_grid["intercept_adjustment"][0]


plt.scatter(
    dataset.grid_points[is_land_subsampled, 0],
    dataset.grid_points[is_land_subsampled, 1],
    c=effects_grid["intercept_adjustment"][1, is_land_subsampled],
    cmap="Blues",
    s=10,
    alpha=0.8,
    vmin=0,
)

## すべての産地の効果比較

4つの主要産地について、同じ効果を並べて比較します。

In [ ]:
def plot_all_origins_single_effect(effect_name: str):
    """全産地について1つの効果を比較プロット"""
    fig, axes = plt.subplots(2, 2, figsize=(12, 10), constrained_layout=True)
    axes = axes.flatten()

    effect_data = effects_grid[effect_name]

    for idx in range(4):  # 4つの主要産地
        ax = axes[idx]
        ax.scatter(
            dataset.grid_points[is_land_subsampled, 0],
            dataset.grid_points[is_land_subsampled, 1],
            c=effect_data[idx, is_land_subsampled],
            cmap="Blues",
            s=10,
            alpha=0.8,
            vmin=0,
            vmax=1,
        )
        ax.scatter(boundary[:, 0], boundary[:, 1], c="grey", s=0.001)
        plt.colorbar(ax.collections[0], ax=ax, label="probability")
        ax.set_title(f"{origins[idx]}")
        ax.set_xlabel("longitude")
        ax.set_ylabel("latitude")

        for spine in ax.spines.values():
            spine.set_linewidth(0.5)

    fig.suptitle(f"Effect: {effect_name}", fontsize=14)
    plt.show()


# 例：距離効果のみで4産地を比較
plot_all_origins_single_effect("distance")

In [ ]:
# 切片効果で4産地を比較
plot_all_origins_single_effect("intercept")

In [ ]:
# 共変量のみの効果（切片を除く）で4産地を比較
plot_all_origins_single_effect("covariates_only")

In [ ]:
# 共変量の総合効果（距離なし）で4産地を比較
plot_all_origins_single_effect("full")

In [ ]:
# 全ての期間（period 0〜4）で独立にモデルを推定し、比較プロット
# 設定は上のセルで定義済みのものを使用

# 各期間のモデルを格納する辞書
all_results = {}
all_datasets = {}
all_grid_probs = {}
all_site_probs = {}

# 5つの期間で独立にモデル推定
for p in range(5):
    print(f"\n{'=' * 60}")
    print(f"Period {p}: {time_periods[p]}")
    print(f"{'=' * 60}")

    # データセット準備（上のセルと同じ設定を使用）
    dataset_p = prepare_distance_prior_dataset(
        preprocessor=preprocessor,
        period=p,
        origins=origins,
        variable_names=variable_names,
        distance_column_names=distance_column_names,
        grid_subsample_ratio=grid_subsample_ratio,
        drop_zero_total_sites=True,
        tau=0.5,
        alpha=1.0,
        source_weights=source_weights,
    )

    # モデル設定（上のセルと同じconfigを再利用）
    config_p = DistancePriorConfig(
        n_iter=config.n_iter,
        burn_in=config.burn_in,
        thinning=config.thinning,
        neighbor_count=config.neighbor_count,
        kernel_lengthscale_by_feature=list(lengthscale.values()),
        kernel_variance_by_feature=list(variance.values()),
        prior_mean_by_feature=list(prior_mean.values()),
        tau=config.tau,
        alpha=config.alpha,
        source_weights=source_weights,
        lambda_fixed=lambda_fixed,
        seed=42,
    )

    # MCMC実行
    results_p = run_mcmc_with_distance_prior(dataset_p, config_p, kernel=None)

    # 事後確率を計算
    site_probs_p = results_p.predict_probabilities(location="sites")
    grid_probs_p = results_p.predict_probabilities(
        location="grid", sample_conditional=False
    )

    # 保存
    all_results[p] = results_p
    all_datasets[p] = dataset_p
    all_grid_probs[p] = grid_probs_p
    all_site_probs[p] = site_probs_p

    print(f"Number of sites: {dataset_p.num_sites()}")
    print(f"Saved samples: {results_p.beta_samples.shape[0]}")

print("\n" + "=" * 60)
print("全期間のモデル推定完了")
print("=" * 60)

## 全期間の比較プロット

各産地について、5つの時期（早期・早々期、前期、中期、後期、晩期）の事後平均確率を横並びで比較します。

In [ ]:
def plot_all_periods_for_origin(origin_index: int):
    """特定の産地について、全期間（0〜4）の事後平均を横並びでプロット"""
    fig, axes = plt.subplots(1, 5, figsize=(25, 5), constrained_layout=True)

    for p_idx, ax in enumerate(axes):
        dataset_p = all_datasets[p_idx]
        grid_probs_p = all_grid_probs[p_idx]

        # 実測比率
        true_ratio_p = dataset_p.counts / dataset_p.counts.sum(axis=1, keepdims=True)
        true_ratio_p = np.nan_to_num(true_ratio_p).T

        # グリッド上の事後平均（陸地のみ）
        sc = ax.scatter(
            dataset.grid_points[is_land_subsampled, 0],
            dataset.grid_points[is_land_subsampled, 1],
            c=grid_probs_p[origin_index, is_land_subsampled],
            cmap="Blues",
            s=10,
            alpha=0.8,
            vmin=0,
            vmax=1,
        )

        # 観測地点の実測比率
        ax.scatter(
            dataset_p.coords[:, 0],
            dataset_p.coords[:, 1],
            c=true_ratio_p[origin_index],
            cmap="Blues",
            s=25,
            edgecolors="black",
            linewidths=0.5,
            alpha=0.8,
            vmin=0,
            vmax=1,
        )

        # 境界
        ax.scatter(boundary[:, 0], boundary[:, 1], c="grey", s=0.001)

        ax.set_title(f"{time_periods[p_idx]} (n={dataset_p.num_sites()})")
        ax.set_xlabel("longitude")
        ax.set_ylabel("latitude")

        for spine in ax.spines.values():
            spine.set_linewidth(0.5)

    # 共通カラーバー
    fig.colorbar(sc, ax=axes, label="posterior mean", shrink=0.8)
    fig.suptitle(f"{origins[origin_index]}産黒曜石の組成比（事後平均）", fontsize=14)
    plt.show()


# 4つの主要産地について比較プロット
for origin_idx in range(4):
    plot_all_periods_for_origin(origin_idx)

### 全産地・全期間の一覧表示

4産地 × 5期間を一度に比較します。

In [ ]:
def plot_all_origins_all_periods():
    """全産地（4行）×全期間（5列）の比較プロット"""
    fig, axes = plt.subplots(4, 5, figsize=(25, 20), constrained_layout=True)

    for origin_idx in range(4):  # 4産地
        for p_idx in range(5):  # 5期間
            ax = axes[origin_idx, p_idx]
            dataset_p = all_datasets[p_idx]
            grid_probs_p = all_grid_probs[p_idx]

            # 実測比率
            true_ratio_p = dataset_p.counts / dataset_p.counts.sum(
                axis=1, keepdims=True
            )
            true_ratio_p = np.nan_to_num(true_ratio_p).T

            # グリッド上の事後平均
            sc = ax.scatter(
                dataset.grid_points[is_land_subsampled, 0],
                dataset.grid_points[is_land_subsampled, 1],
                c=grid_probs_p[origin_idx, is_land_subsampled],
                cmap="Blues",
                s=5,
                alpha=0.8,
                vmin=0,
                vmax=1,
            )

            # 観測地点の実測比率
            ax.scatter(
                dataset_p.coords[:, 0],
                dataset_p.coords[:, 1],
                c=true_ratio_p[origin_idx],
                cmap="Blues",
                s=15,
                edgecolors="black",
                linewidths=0.3,
                alpha=0.8,
                vmin=0,
                vmax=1,
            )

            # 境界
            ax.scatter(boundary[:, 0], boundary[:, 1], c="grey", s=0.001)

            # 行ラベル（産地名）は左端のみ
            if p_idx == 0:
                ax.set_ylabel(f"{origins[origin_idx]}", fontsize=12)

            # 列ラベル（期間名）は上端のみ
            if origin_idx == 0:
                ax.set_title(f"{time_periods[p_idx]}", fontsize=10)

            ax.set_xticks([])
            ax.set_yticks([])

            for spine in ax.spines.values():
                spine.set_linewidth(0.1)

    # 共通カラーバー
    fig.colorbar(sc, ax=axes, label="posterior mean probability", shrink=0.6, pad=0.02)
    fig.suptitle("黒曜石産地組成比の時代変化（事後平均）", fontsize=16, y=1.01)
    plt.show()


plot_all_origins_all_periods()

### 各期間の遺跡数サマリー

In [ ]:
# 各期間の遺跡数と産地ごとの総出土数をサマリー
print("=" * 70)
print("各期間のサマリー")
print("=" * 70)

for p_idx in range(5):
    dataset_p = all_datasets[p_idx]
    counts_sum = dataset_p.counts.sum(axis=0)  # 各産地の総出土数

    print(f"\n{time_periods[p_idx]} (period={p_idx})")
    print(f"  遺跡数: {dataset_p.num_sites()}")
    print(f"  総出土数: {int(dataset_p.counts.sum())}")
    print("  産地別出土数:")
    for k, origin in enumerate(origins):
        print(
            f"    {origin}: {int(counts_sum[k])} ({counts_sum[k] / counts_sum.sum() * 100:.1f}%)"
        )

### 効果の期間・産地比較プロット

指定した効果について、複数の期間と産地を柔軟に比較できる関数です。

In [ ]:
# 各期間の効果分解を事前計算
all_effects = {p: all_results[p].decompose_effects(location="grid") for p in range(5)}

# 利用可能な効果名を表示
print("利用可能な効果名:")
for key in all_effects[0].keys():
    print(f"  - {key}")

In [ ]:
def plot_effect_by_periods_and_origins(
    effect_name: str,
    periods: list[int],
    origin_indices: list[int],
    scatter: bool = False,
    cmap: str = "Blues",
    vcenter: float = None,
):
    """指定した効果を、期間×産地のグリッドで比較プロット

    Parameters
    ----------
    effect_name : str
        プロットする効果名（"distance", "intercept", "full"など）
    periods : list[int]
        プロットする期間のリスト（例: [0, 2, 3]）
    origin_indices : list[int]
        プロットする産地のインデックスリスト（例: [0, 1]は神津島と信州）
    cmap : str
        カラーマップ
    vcenter : float
        カラーバーの中心値
    """
    n_periods = len(periods)
    n_origins = len(origin_indices)

    fig, axes = plt.subplots(
        n_origins,
        n_periods,
        figsize=(5 * n_periods, 5 * n_origins),
        constrained_layout=True,
        squeeze=False,
    )

    # プロットするデータの最大値と最小値を取得

    max_val = -np.inf
    min_val = np.inf
    for p_idx in periods:
        effect_data = all_effects[p_idx][effect_name]
        max_val = max(max_val, effect_data.max())
        min_val = min(min_val, effect_data.min())

    print(f"max_val: {max_val}, min_val: {min_val}")

    if vcenter is not None:
        # vcenterを中心に、同じ幅になるようにする
        max_val_diff = abs(max_val - vcenter)
        min_val_diff = abs(vcenter - min_val)
        diff = max(max_val_diff, min_val_diff)
        max_val = vcenter + diff
        min_val = vcenter - diff

        norm = TwoSlopeNorm(vmin=min_val, vcenter=vcenter, vmax=max_val)
        cmap = "RdBu"
    else:
        norm = None

    for row_idx, origin_idx in enumerate(origin_indices):
        for col_idx, p_idx in enumerate(periods):
            ax = axes[row_idx, col_idx]

            # 効果データを取得
            effect_data = all_effects[p_idx][effect_name]

            # グリッド上の効果をプロット
            sc = ax.scatter(
                dataset.grid_points[is_land_subsampled, 0],
                dataset.grid_points[is_land_subsampled, 1],
                c=effect_data[origin_idx, is_land_subsampled],
                cmap=cmap,
                s=8,
                alpha=0.8,
                norm=norm,
            )

            # 観測地点をプロット（fullの場合のみ実測比率も表示）
            if effect_name == "full" or scatter:
                dataset_p = all_datasets[p_idx]
                true_ratio_p = dataset_p.counts / dataset_p.counts.sum(
                    axis=1, keepdims=True
                )
                true_ratio_p = np.nan_to_num(true_ratio_p).T
                ax.scatter(
                    dataset_p.coords[:, 0],
                    dataset_p.coords[:, 1],
                    c=true_ratio_p[origin_idx],
                    cmap=cmap,
                    norm=norm,
                    s=20,
                    edgecolors="black",
                    linewidths=0.3,
                    alpha=0.8,
                )

            # 境界
            ax.scatter(boundary[:, 0], boundary[:, 1], c="grey", s=0.001)

            # ラベル
            if row_idx == 0:
                ax.set_title(f"{time_periods[p_idx]}", fontsize=11)
            if col_idx == 0:
                ax.set_ylabel(f"{origins[origin_idx]}", fontsize=11)

            ax.set_xticks([])
            ax.set_yticks([])

            for spine in ax.spines.values():
                spine.set_linewidth(0.3)

    # 共通カラーバー
    fig.colorbar(sc, ax=axes, label="probability", shrink=0.7, pad=0.02)
    fig.suptitle(f"Effect: {effect_name}", fontsize=14, y=1.01)
    plt.show()

In [ ]:
# 使用例：神津島と信州について、全期間の"full"効果を比較
plot_effect_by_periods_and_origins(
    "full", periods=[0, 1, 2, 3, 4], origin_indices=[0, 1]
)

In [ ]:
# 使用例：全産地について、中期と後期の"distance"効果を比較
plot_effect_by_periods_and_origins(
    "distance", periods=[0, 1, 2, 3, 4], origin_indices=[0, 1]
)

In [ ]:
# 使用例：神津島のみ、全期間で"intercept_adjustment"（データによる調整）を比較
plot_effect_by_periods_and_origins(
    "intercept_adjustment",
    periods=[0, 1, 2, 3, 4],
    origin_indices=[0, 1, 2, 3],
    cmap="RdBu",
    vcenter=0.2,
    scatter=True,
)